In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile
import os



In [ ]:
zip_path = '/content/Dataset _S.zip'
extraction_path = '/mnt/data/Dataset/'


In [ ]:
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)


In [ ]:
print("Extracted files:", os.listdir(extraction_path))

Extracted files: ['Dataset']


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
img_height, img_width = 224, 224  # Size expected by many models
batch_size = 32


In [ ]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.4)  #

In [ ]:
train_gen = datagen.flow_from_directory(
    extraction_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    subset='training')


Found 1440 images belonging to 1 classes.


In [ ]:
validation_gen = datagen.flow_from_directory(
    extraction_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    subset='validation')


Found 960 images belonging to 1 classes.


In [ ]:
val_size = int(0.5 * validation_gen.samples)
test_size = validation_gen.samples - val_size
from tensorflow.keras import layers, models


In [ ]:
cnn_model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(train_gen.class_indices), activation='softmax')])




/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cnn_model.fit(train_gen, validation_data=validation_gen, epochs=10)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/losses/losses.py:27: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored

15/45 ━━━━━━━━━━━━━━━━━━━━ 1:59 4s/step - accuracy: 1.0000 - loss: 0.0000e+00

KeyboardInterrupt: 

In [ ]:
from tensorflow.keras.applications import VGG16

In [ ]:
vgg16_base = VGG16(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')
vgg16_base.trainable = False


In [ ]:
transfer_model = models.Sequential([
    vgg16_base,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(train_gen.class_indices), activation='softmax')])


In [ ]:
transfer_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
transfer_model.fit(train_gen, validation_data=validation_gen, epochs=10)


Epoch 1/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 1528s 34s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 1506s 34s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 1528s 34s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 1555s 34s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 1523s 34s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 1547s 34s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 1579s 34s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 150

In [ ]:
from tensorflow.keras.applications import ResNet50

In [ ]:
resnet_base = ResNet50(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')
resnet_base.trainable = False


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
resnet_model = models.Sequential([
    resnet_base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(train_gen.class_indices), activation='softmax')])


In [ ]:
resnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
resnet_model.fit(train_gen, validation_data=validation_gen, epochs=10)
augmented_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.4)


Epoch 1/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 468s 10s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 516s 11s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 480s 10s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 453s 10s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 505s 10s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 454s 10s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 452s 10s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 472s 11s/s

In [ ]:
test_gen = datagen.flow_from_directory(
    extraction_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    subset='validation',  # Remaining 20% as the test set
    shuffle=False)


Found 960 images belonging to 1 classes.


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [ ]:
def evaluate_model(model, test_gen):
    test_gen.reset()
    predictions = model.predict(test_gen)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_gen.classes
    class_labels = list(test_gen.class_indices.keys())



In [ ]:
import numpy as np
# Import necessary function in the same cell where it's being used.
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import true_classes, predicted_classes, class_labels

ImportError: cannot import name 'true_classes' from 'sklearn.metrics' (/usr/local/lib/python3.10/dist-packages/sklearn/metrics/__init__.py)

In [ ]:
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
cm = confusion_matrix(true_classes, predicted_classes)

print("Classification Report:\n", report)
print("Confusion Matrix:\n", cm)


NameError: name 'true_classes' is not defined

In [ ]:
print("CNN Model Evaluation:")
cnn_report = classification_report(true_classes, predicted_classes, target_names=class_labels)
cnn_cm = confusion_matrix(true_classes, predicted_classes)
print(cnn_report)



CNN Model Evaluation:


NameError: name 'true_classes' is not defined

In [ ]:
print("\nTransfer Learning Model (VGG16) Evaluation:")
vgg16_report, vgg16_cm = evaluate_model(transfer_model, test_gen)


In [ ]:
print("\nResNet Model Evaluation:")
resnet_report, resnet_cm = evaluate_model(resnet_model, test_gen)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.callbacks import History


In [ ]:
def plot_confusion_matrix(cm, class_labels, title='Confusion Matrix'):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title(title)
    plt.show()


In [ ]:
def plot_training_history(history, title):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs_range = range(len(acc))

    plt.figure(figsize=(14, 6))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title(f'{title} - Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title(f'{title} - Loss')
    plt.show()


In [ ]:
print("CNN Confusion Matrix:")
plot_confusion_matrix(cnn_cm, class_labels, title='CNN Confusion Matrix')


CNN Confusion Matrix:


NameError: name 'plot_confusion_matrix' is not defined

In [ ]:
print("VGG16 Confusion Matrix:")
plot_confusion_matrix(vgg16_cm, class_labels, title='VGG16 Confusion Matrix')


VGG16 Confusion Matrix:


NameError: name 'plot_confusion_matrix' is not defined

In [ ]:
print("ResNet Confusion Matrix:")
plot_confusion_matrix(resnet_cm, class_labels, title='ResNet Confusion Matrix')


ResNet Confusion Matrix:


NameError: name 'plot_confusion_matrix' is not defined

In [ ]:
plot_training_history(cnn_history, "CNN Model")
plot_training_history(vgg16_history, "VGG16 Transfer Model")
plot_training_history(resnet_history, "ResNet Model")


NameError: name 'cnn_history' is not defined

Try this code

In [ ]:


from google.colab import drive
import zipfile
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import ResNet50
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import true_classes, predicted_classes, class_labels
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.callbacks import History
drive.mount('/content/drive')


zip_path = '/content/Dataset _S.zip'
extraction_path = '/mnt/data/Dataset/'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

print("Extracted files:", os.listdir(extraction_path))
img_height, img_width = 224, 224  # Size expected by many models
batch_size = 32

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.4)  #
train_gen = datagen.flow_from_directory(
    extraction_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    subset='training')

validation_gen = datagen.flow_from_directory(
    extraction_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    subset='validation')

val_size = int(0.5 * validation_gen.samples)
test_size = validation_gen.samples - val_size

cnn_model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(train_gen.class_indices), activation='softmax')])



cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cnn_history = cnn_model.fit(train_gen, validation_data=validation_gen, epochs=10)

vgg16_base = VGG16(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')
vgg16_base.trainable = False

transfer_model = models.Sequential([
    vgg16_base,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(train_gen.class_indices), activation='softmax')])

transfer_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
vgg16_history = transfer_model.fit(train_gen, validation_data=validation_gen, epochs=10)

resnet_base = ResNet50(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')
resnet_base.trainable = False

resnet_model = models.Sequential([
    resnet_base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(train_gen.class_indices), activation='softmax')])

resnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
resnet_history = resnet_model.fit(train_gen, validation_data=validation_gen, epochs=10)
augmented_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.4)

test_gen = datagen.flow_from_directory(
    extraction_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    subset='validation',  # Remaining 20% as the test set
    shuffle=False)

def evaluate_model(model, test_gen):
    test_gen.reset()
    predictions = model.predict(test_gen)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_gen.classes
    class_labels = list(test_gen.class_indices.keys())
    report = classification_report(true_classes, predicted_classes, target_names=class_labels)
    cm = confusion_matrix(true_classes, predicted_classes)
    return report, cm

# Import necessary function in the same cell where it's being used.
test_gen.reset()
predictions = cnn_model.predict(test_gen)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_gen.classes
class_labels = list(test_gen.class_indices.keys())
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
cm = confusion_matrix(true_classes, predicted_classes)

print("Classification Report:\n", report)
print("Confusion Matrix:\n", cm)

print("CNN Model Evaluation:")
cnn_report, cnn_cm = evaluate_model(cnn_model, test_gen)
print(cnn_report)


print("\nTransfer Learning Model (VGG16) Evaluation:")
vgg16_report, vgg16_cm = evaluate_model(transfer_model, test_gen)

print("\nResNet Model Evaluation:")
resnet_report, resnet_cm = evaluate_model(resnet_model, test_gen)

def plot_confusion_matrix(cm, class_labels, title='Confusion Matrix'):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title(title)
    plt.show()

def plot_training_history(history, title):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs_range = range(len(acc))

    plt.figure(figsize=(14, 6))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title(f'{title} - Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title(f'{title} - Loss')
    plt.show()

print("CNN Confusion Matrix:")
plot_confusion_matrix(cnn_cm, class_labels, title='CNN Confusion Matrix')

print("VGG16 Confusion Matrix:")
plot_confusion_matrix(vgg16_cm, class_labels, title='VGG16 Confusion Matrix')

print("ResNet Confusion Matrix:")
plot_confusion_matrix(resnet_cm, class_labels, title='ResNet Confusion Matrix')

plot_training_history(cnn_history, "CNN Model")
plot_training_history(vgg16_history, "VGG16 Transfer Model")
plot_training_history(resnet_history, "ResNet Model")

